In [5]:
import os
import cv2
import shutil
import base64
import requests
import numpy as np
import mediapipe as mp
os.makedirs("_output", exist_ok=True)

In [6]:
FORGE_INSTANCE_URL = "http://127.0.0.1:1234" # "http://127.0.0.1:1234" / "https://*-1234.proxy.runpod.net"

In [7]:
def upscale_image(img_inp_path, img_out_path):

    with open(img_inp_path, "rb") as f_img_inp:
        img_inp_base64 = base64.b64encode(f_img_inp.read()).decode('utf-8')
    # img_h, img_w, _ = cv2.imread(img_inp_path).shape

    payload = {
        "resize_mode": 0,                  # Sets the resize mode: 0 to upscale by upscaling_resize amount, 1 to upscale up to upscaling_resize_h x upscaling_resize_w.
        "show_extras_results": False,      # Should the backend return the generated image?
        "gfpgan_visibility": 0.35,         # Sets the visibility of GFPGAN, values should be between 0 and 1.
        "codeformer_visibility": 0,        # Sets the visibility of CodeFormer, values should be between 0 and 1.
        "codeformer_weight": 0,            # Sets the weight of CodeFormer, values should be between 0 and 1.
        "upscaling_resize": 4,             # By how much to upscale the image, only used when resize_mode=0.
        "upscaling_resize_w": 512,         # Target width for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_resize_h": 512,         # Target height for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_crop": True,            # Should the upscaler crop the image to fit in the chosen size?
        "upscaler_1": "R-ESRGAN 4x+",      # The name of the main upscaler to use
        "upscaler_2": "None",              # The name of the secondary upscaler to use
        "extras_upscaler_2_visibility": 0, # Sets the visibility of secondary upscaler, values should be between 0 and 1.
        "upscale_first": False,            # Should the upscaler run before restoring faces?
        "image": img_inp_base64            # Image to work on, must be a Base64 string containing the image's data.
    }

    def request_forge_instance(payload, img_out_path):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/extra-single-image", json=payload) as req:
                with open(img_out_path, "wb") as f:
                    f.write(base64.b64decode(req.json()["image"]))
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"⚠️ req.json(): {req.json()}")
    request_forge_instance(payload, img_out_path)

def resize_image(img_inp_path, img_out_path, new_width=1080):
    img = cv2.imread(img_inp_path)
    h, w, _ = img.shape
    new_height = int(h * new_width / w)
    img_resized = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
    cv2.imwrite(img_out_path, img_resized)

def create_facemask(img_inp_path, img_keypoints_path, img_mask_path, expand_scale_factor=1.2):
    def get_ls_keypoints(img_inp_path, img_out_path, expand_scale_factor):
        def expand_polygon(ls_keypoints, expand_scale_factor):
            points = np.array(ls_keypoints, dtype=np.float32)                      # Convert keypoints to a NumPy array
            centroid = np.mean(points, axis=0)                                     # Calculate the centroid of the polygon
            expanded_points = centroid + (points - centroid) * expand_scale_factor # Translate points to origin (subtract centroid), scale, and translate back
            expanded_points = expanded_points.astype(np.int32)                     # Convert back to integer coordinates for OpenCV
            return expanded_points.tolist()
        # Read image
        img_raw = cv2.imread(img_inp_path)
        img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
        h, w, _ = img_raw.shape
        # MediaPipe: Extract face landmarks
        with mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.2) as face_mesh:
            mp_res = face_mesh.process(img_rgb)
            if mp_res.multi_face_landmarks:
                for facelandmarks in mp_res.multi_face_landmarks:
                    # Landmark indices
                    ls_indices = [10,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172,58,132,93,234,127,162,21,54,103,67,109]
                    # Get coordinates
                    ls_keypoints = []
                    for idx in ls_indices:
                        ls_keypoints.append(
                            (int(facelandmarks.landmark[idx].x * w), int(facelandmarks.landmark[idx].y * h)) # 1 keypoint (x,y)
                        )
                    # Expand polygon
                    ls_keypoints_expanded = expand_polygon(ls_keypoints, expand_scale_factor)
                    # Draw points
                    DRAW_THICKNESS = 18
                    cv2.polylines(img_raw, [np.array(ls_keypoints, dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=DRAW_THICKNESS)
                    cv2.polylines(img_raw, [np.array(ls_keypoints_expanded, dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=DRAW_THICKNESS)
                    for keypoint in ls_keypoints:
                        cv2.circle(img_raw, keypoint, DRAW_THICKNESS, (0, 255, 0), -1)
                    cv2.imwrite(img_out_path, img_raw)
                    # Return
                    return ls_keypoints_expanded
        raise ValueError("⚠️ No faces detected")
    
    ls_keypoints = get_ls_keypoints(img_inp_path, img_keypoints_path, expand_scale_factor)
    h, w, _ = cv2.imread(img_inp_path).shape

    # Convert to numpy array (required format for OpenCV fillPoly)
    ls_keypoints = np.array(ls_keypoints, np.int32)
    ls_keypoints = ls_keypoints.reshape((-1, 1, 2)) # shape (n,1,2)
    # Create a blank black image (size depends on your image)
    img_mask = np.zeros((h, w, 3), dtype=np.uint8)
    # Fill the polygon with white
    cv2.fillPoly(img_mask, [ls_keypoints], (255, 255, 255))
    cv2.imwrite(img_mask_path, img_mask)

def inpaint_image(img_inp_path, img_mask_path, img_out_path, sd_params):

    with open(img_inp_path, "rb") as f_img_inp, open(img_mask_path, "rb") as f_img_msk:
        img_inp_base64 = base64.b64encode(f_img_inp.read()).decode('utf-8')
        img_msk_base64 = base64.b64encode(f_img_msk.read()).decode('utf-8')
    img_h, img_w, _ = cv2.imread(img_inp_path).shape

    payload = {
        "prompt": sd_params["prompt"],
        "negative_prompt": "",
        "styles": [],
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": sd_params["sampler"],
        "scheduler": sd_params["scheduler"],
        "batch_size": 1,
        "n_iter": 1,
        "steps": sd_params["steps"],
        "cfg_scale": sd_params["cfg"],
        "distilled_cfg_scale": 3.5,
        "width": img_w,
        "height": img_h,
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "eta": 0,
        "denoising_strength": sd_params["denoising"],
        "s_min_uncond": 0.0,
        "s_churn": 0.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "s_noise": 1.0,
        "override_settings": {"sd_model_checkpoint": sd_params["model"], "CLIP_stop_at_last_layers": 1},
        "override_settings_restore_afterwards": False,
        # "refiner_checkpoint": "string",
        "refiner_switch_at": 0,
        "disable_extra_networks": False,
        # "firstpass_image": "string",
        "comments": {},
        "init_images": [img_inp_base64],
        "resize_mode": 0,
        "image_cfg_scale": 1.5,
        "mask": img_msk_base64,
        "mask_blur_x": 4,
        "mask_blur_y": 4,
        "mask_blur": 4,
        "mask_round": True,
        "inpainting_fill": 1,
        "inpaint_full_res": 0,
        "inpaint_full_res_padding": 32,
        "inpainting_mask_invert": 0,
        "initial_noise_multiplier": 1.0,
        # "latent_mask": "string",
        # "force_task_id": "string",
        "hr_distilled_cfg": 3.5,
        "sampler_index": "Euler",
        "include_init_images": False,
        "script_name": None,
        "script_args": [],
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
        # "infotext": "string"
    }

    def request_forge_instance(payload, savepath):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/img2img", json=payload) as req:
                with open(savepath, "wb") as f:
                    f.write(base64.b64decode(req.json()["images"][0]))
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"⚠️ req.json(): {req.json()}")
    request_forge_instance(payload, img_out_path)

In [8]:
PATH_INPUT     = "_test/img_0.jpg" # r"D:\DRIVE\Drive\ig_xouuuus\IG_Girls_Photos\xouuuus\3.jpg" # "_test/img_0.jpg"
PATH_UPSCALED  = "_output/1_upscaled.jpg"
PATH_KEYPOINTS = "_output/2_keypoints.jpg"
PATH_FACEMASK  = "_output/2_facemask.jpg"
GENERATION_VARIANTS = [
    {
        "WIDTH": 1080,
        "PATH_UPSCALED_RESIZED": "_output/3_resized_1080.jpg",
        "PATH_FACEMASK_RESIZED": "_output/3_facemask_1080.jpg",
        "PATH_OUTPUT": "_output/4_output_1080.jpg",
        "SD_PARAMS": {
            "prompt": "best quality, masterpiece, ultra high res, photorealistic, 1girl, <lora:LORA_KoreaDL:0.5>",
            "cfg": 5, "denoising": 0.6, "steps": 20, "sampler": "Euler a", "scheduler": "Automatic", "model": "MJX_V07",
        }
    },
    {
        "WIDTH": 960,
        "PATH_UPSCALED_RESIZED": "_output/3_resized_960.jpg",
        "PATH_FACEMASK_RESIZED": "_output/3_facemask_960.jpg",
        "PATH_OUTPUT": "_output/4_output_960.jpg",
        "SD_PARAMS": {
            "prompt": "best quality, masterpiece, ultra high res, photorealistic, 1girl, <lora:LORA_KoreaDL:0.5>",
            "cfg": 5, "denoising": 0.6, "steps": 20, "sampler": "Euler a", "scheduler": "Automatic", "model": "MJX_V07",
        }
    },
]

# ----- 0
shutil.copy2(PATH_INPUT, f"_output/0_input{os.path.splitext(PATH_INPUT)[1]}")
# ----- 1
upscale_image(PATH_INPUT, PATH_UPSCALED)
# ----- 2
create_facemask(PATH_UPSCALED, PATH_KEYPOINTS, PATH_FACEMASK, expand_scale_factor=1.4)
# ----- 3
for variant in GENERATION_VARIANTS:
    resize_image(PATH_UPSCALED, variant["PATH_UPSCALED_RESIZED"], new_width=variant["WIDTH"])
    resize_image(PATH_FACEMASK, variant["PATH_FACEMASK_RESIZED"], new_width=variant["WIDTH"])
# ----- 4
for variant in GENERATION_VARIANTS:
    inpaint_image(variant["PATH_UPSCALED_RESIZED"], variant["PATH_FACEMASK_RESIZED"], variant["PATH_OUTPUT"], variant["SD_PARAMS"])